In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from datetime import datetime


# 获取数据集

In [2]:
# 读取Facebook位置数据集
print("读取Facebook位置数据集...")
fb_train = pd.read_csv('C:\\Users\\19942\\Desktop\\wangdao\\code\\python_ml\\data\\FBlocation\\train.csv')


# 相比x，y，准确性和地点id，使用时间特征更能提高模型准确性，对时间进行细分

# 将time列转换为年月日时分秒格式
print("将time列转换为年月日时分秒格式...")
# 假设time以秒为单位，转换为时间格式（to_datetime)，并添加为新列time_formatted
fb_train['time_formatted'] = pd.to_datetime(fb_train['time'], unit='s')

# 从time_formatted提取以下特征:
# 星期几(dt.dayofweek)、小时(dt.hour)和天(dt.day)
print("从时间中提取星期几、小时和分钟作为新特征...")
fb_train['weekday'] = fb_train['time_formatted'].dt.dayofweek  # 星期几 (0-6, 0是星期一)
fb_train['hour'] = fb_train['time_formatted'].dt.hour  # 小时 (0-23)
fb_train['day'] = fb_train['time_formatted'].dt.day  # 天

# 显示数据集基本信息
print("数据集基本信息：")
print(f"样本数量: {fb_train.shape[0]}")
print(f"特征数量: {fb_train.shape[1]}")
print('数据集fb_train的类型：',type(fb_train))
print("\n数据集的前5行:")
print(fb_train.head(5))

读取Facebook位置数据集...
将time列转换为年月日时分秒格式...
从时间中提取星期几、小时和分钟作为新特征...
数据集基本信息：
样本数量: 29118021
特征数量: 10
数据集fb_train的类型： <class 'pandas.core.frame.DataFrame'>

数据集的前5行:
   row_id       x       y  accuracy    time    place_id      time_formatted  \
0       0  0.7941  9.0809        54  470702  8523065625 1970-01-06 10:45:02   
1       1  5.9567  4.7968        13  186555  1757726713 1970-01-03 03:49:15   
2       2  8.3078  7.0407        74  322648  1137537235 1970-01-04 17:37:28   
3       3  7.3665  2.5165        65  704587  6567393236 1970-01-09 03:43:07   
4       4  4.0961  1.1307        31  472130  7440663949 1970-01-06 11:08:50   

   weekday  hour  day  
0        1    10    6  
1        5     3    3  
2        6    17    4  
3        4     3    9  
4        1    11    6  


# 数据预处理

In [ ]:


# 检查缺失值
print("\n检查缺失值:")
print(fb_train.isnull().sum())      # 如果有缺失，用SimpleImputer替换


# 对x和y的数值范围进行筛选
# 筛选出x在1.0到1.25之间，y在2.5到2.75之间的数据
filtered_fb_train=fb_train.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")
print(f"筛选后的样本数量: {filtered_fb_train.shape[0]}")
print(f"筛选前的样本数量: {fb_train.shape[0]}")
print(f"筛选比例: {filtered_fb_train.shape[0]/fb_train.shape[0]:.2%}")

# 对place_id进行筛选，过滤掉出现次数小于3的place（冷门地点）
# 统计每个place_id的出现次数
print("\n统计每个place_id的出现次数...")
place_counts = filtered_fb_train['place_id'].value_counts()

# 找出出现次数大于3的place_id
valid_places = place_counts[place_counts > 3].index

# 只保留出现次数大于3的place_id对应的数据
print("去除place_id出现次数小于等于3的数据...")
filtered_fb_train = filtered_fb_train[filtered_fb_train['place_id'].isin(valid_places)]


print(f"去除后的样本数量: {filtered_fb_train.shape[0]}")
print(f"去除的样本比例: {1 - filtered_fb_train.shape[0]/fb_train.shape[0]:.2%}")
print(f"剩余的不同place_id数量: {filtered_fb_train['place_id'].nunique()}")

print('\n过滤后的训练集（filter_fb_train）的部分信息：\n',filtered_fb_train.head())


检查缺失值:
row_id            0
x                 0
y                 0
accuracy          0
time              0
place_id          0
time_formatted    0
weekday           0
hour              0
day               0
dtype: int64
筛选后的样本数量: 17710
筛选前的样本数量: 29118021
筛选比例: 0.06%

统计每个place_id的出现次数...
去除place_id出现次数小于等于3的数据...
去除后的样本数量: 16918
去除的样本比例: 99.94%
剩余的不同place_id数量: 239

过滤后的训练集（filter_fb_train）的部分信息：
       row_id       x       y  accuracy    time    place_id  \
600      600  1.2214  2.7023        17   65380  6683426742   
957      957  1.1832  2.6891        58  785470  6683426742   
4345    4345  1.1935  2.6550        11  400082  6889790653   
4735    4735  1.1452  2.6074        49  514983  6822359752   
5580    5580  1.0089  2.7287        19  732410  1527921905   

          time_formatted  weekday  hour  day  
600  1970-01-01 18:09:40        3    18    1  
957  1970-01-10 02:11:10        5     2   10  
4345 1970-01-05 15:08:02        0    15    5  
4735 1970-01-06 23:03:03        1    

# 划分训练集和验证集

In [4]:

# 提取特征和标签，加入新提取的时间特征，然后指定标签(place_id)

X = filtered_fb_train[['x', 'y', 'accuracy', 'weekday', 'hour', 'day']]  # 特征
y = filtered_fb_train['place_id']  # 把place_id作为标签
# 划分训练集和验证集
print("\n划分训练集和验证集...")
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('测试集X_train的前5行：\n',X_train.head(5))

# 特征标准化 - 使用训练集拟合标准化器，然后分别转换训练集和验证集
print("\n对特征进行标准化处理...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 对训练集进行fit和transform
X_val_scaled = scaler.transform(X_val)  # 对验证集只进行transform

# 打印数据集划分结果
print("Facebook位置数据集划分结果：")
print(f"训练集样本数量: {X_train.shape[0]}")
print(f"验证集样本数量: {X_val.shape[0]}")
print(f"特征数量: {X_train.shape[1]}")

# 统计标签数量
print("\n标签(place_id)的数量(unique):", y.nunique())

print("\n数据预处理完成，可以进行KNN训练")

X_train_scaled[0:5]


划分训练集和验证集...
测试集X_train的前5行：
                x       y  accuracy  weekday  hour  day
5409451   1.0850  2.6961        62        5     0    3
24956130  1.1627  2.6654        43        1    13    6
25705982  1.0108  2.5411       165        5     0    3
8937451   1.1814  2.5099        38        3     5    8
18299121  1.2061  2.5105        57        3    21    8

对特征进行标准化处理...
Facebook位置数据集划分结果：
训练集样本数量: 13534
验证集样本数量: 3384
特征数量: 6

标签(place_id)的数量(unique): 239

数据预处理完成，可以进行KNN训练


array([[-0.48293676,  0.90291955, -0.17915424,  1.13044561, -1.65117484,
        -0.76794871],
       [ 0.52467098,  0.46434201, -0.35200665, -1.25794776,  0.2182894 ,
         0.33718539],
       [-1.44515676, -1.31139703,  0.75788777,  1.13044561, -1.65117484,
        -0.76794871],
       [ 0.76717117, -1.75711752, -0.39749413, -0.06375108, -0.93215013,
         1.07394146],
       [ 1.0874789 , -1.74854597, -0.22464172, -0.06375108,  1.36872893,
         1.07394146]])

# 训练、评估模型

In [5]:
# 导入KNN分类器和评估指标
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

print("开始训练KNN模型...")

# 记录开始时间
start_time = time.time()

# 创建KNN分类器，使用默认K=5
knn = KNeighborsClassifier(n_neighbors=5)

# 训练模型
knn.fit(X_train_scaled, y_train)

# 记录结束时间
end_time = time.time()
print(f"KNN模型训练和评估总耗时: {end_time - start_time:.2f} 秒")


# 在验证集上进行预测
y_pred = knn.predict(X_val_scaled)

# 计算准确率
accuracy = accuracy_score(y_val, y_pred)
print(f"KNN模型准确率: {accuracy:.4f}")


print("\nKNN模型训练和评估完成！")


开始训练KNN模型...
KNN模型训练和评估总耗时: 0.02 秒
KNN模型准确率: 0.4805

KNN模型训练和评估完成！


# 使用网格搜索，对超参数进行调整

In [6]:
# 导入GridSearchCV和交叉验证相关工具
from sklearn.model_selection import GridSearchCV


print("开始使用GridSearchCV进行KNN参数优化...")


# 设置开始时间
start_time = time.time()

# 定义参数网格
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance']
}

# 创建KNN分类器
# 会爆出警告，因为可能有一个特征的样本太少，无法计算邻居
knn = KNeighborsClassifier()

# 创建GridSearchCV对象
grid_search = GridSearchCV(
    estimator=knn,              # 要优化的基础KNN分类器
    param_grid=param_grid,      # 超参数搜索空间
    cv=3,                       # 3折交叉验证
    scoring='accuracy',         # 以准确率accuracy作为评估指标
    n_jobs=-1                   # 并行使用所有可以使用的CPU核心(-1)
)

# 在训练集上拟合网格搜索，这个过程会对特征自动训练不需要knn.fit
grid_search.fit(X_train_scaled, y_train)

# 获取最佳参数和最佳得分
best_params = grid_search.best_params_
best_score = grid_search.best_score_


# 计算结束时间
end_time = time.time()
time_used = end_time - start_time

# 输出结果
print(f"\n网格搜索完成，耗时: {time_used:.2f} 秒")
print(f"最佳参数: {best_params}")
print(f"交叉验证最佳得分: {best_score:.4f}")


开始使用GridSearchCV进行KNN参数优化...


d:\py\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



网格搜索完成，耗时: 6.96 秒
最佳参数: {'n_neighbors': 11, 'weights': 'distance'}
交叉验证最佳得分: 0.4885


In [7]:
# 使用最佳参数构建模型
# best_knn = KNeighborsClassifier(**best_params)
# best_knn.fit(X_train_scaled, y_train)

# 不需要重新构建和训练模型，GridSearchCV已经用最优参数在fit时训练好了模型
best_knn = grid_search.best_estimator_

# 在验证集上评估最佳模型
y_pred_best = best_knn.predict(X_val_scaled)
best_accuracy = accuracy_score(y_val, y_pred_best)
print(f"最佳模型在验证集上的准确率: {best_accuracy:.4f}")

最佳模型在验证集上的准确率: 0.5003
